In [1]:
import time
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import re
import matplotlib.pyplot as plt
import pytz
from datetime import datetime
# Start time
start_time = time.time()

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


# Define the same model architecture
# Define ResNetBlock1
class ResNetBlock1(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResNetBlock1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.identity_downsample = nn.Conv2d(in_channels, out_channels, kernel_size=1,stride=1) if in_channels != out_channels else None

    def forward(self, x):
        identity = x
        if self.identity_downsample is not None:
            identity = self.identity_downsample(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        out = F.relu(out)
        return out


# Define ResNetBlock2
class ResNetBlock2(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResNetBlock2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else None

    def forward(self, x):
        identity = x
        if self.conv3 is not None:
            identity = self.conv3(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        out = F.relu(out)
        return out


# Define the entire network
class FinalCNN(nn.Module):
    def __init__(self):
        super(FinalCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = ResNetBlock1(in_channels=64, out_channels=64)
        self.layer2 = ResNetBlock2(in_channels=64, out_channels=128)
        self.layer3 = ResNetBlock1(in_channels=128, out_channels=256)
        self.layer4 = ResNetBlock2(in_channels=256, out_channels=128)
        self.avgpool = nn.AdaptiveAvgPool2d((2, 2))
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512, 1024)
        self.fc2 = nn.Linear(1024, 128)
        #self.dropout = nn.Dropout(0.5)
        self.fc3 = nn.Linear(128, 128)  # 64 for real and 64 for imaginary

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = self.dropout(x)
        x = torch.tanh(self.fc3(x))  # Clamping the output to be within [-1, 1]
        return x


# Function to process image file
def process_image_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    images = []
    image = []
    for line in lines:
        row = [int(pixel) for pixel in line.strip().split(',')]
        image.append(row)
        if len(image) == 64:
            images.append(image)
            image = []
    images = np.array(images, dtype=np.float32)
    images = images.reshape(-1, 1, 64, 64)  # -1 to automatically determine the number of images
    return images


file_path1 = 'Simulator_Data/sample135_matrix.txt'  
images = process_image_file(file_path1)




# Custom dataset class
class ImageDataset(Dataset):
    def __init__(self, images):
        self.images = torch.tensor(images, dtype=torch.float32)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]

        return image


dataset = ImageDataset(images )
test_loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Accuracy calculation function



# Load the saved model
model_path = 'Simulator_trained/model_28_1039.pth'  # replace with your saved model path
model = FinalCNN().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()


# Run the model on the new test dataset

predicted_real = []
predicted_imag = []



with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        outputs_real = outputs[:, :64]
        outputs_imag = outputs[:, 64:]


        predicted_real.append(outputs_real.cpu().numpy())
        predicted_imag.append(outputs_imag.cpu().numpy())


# Convert collected lists to numpy arrays

predicted_real = np.concatenate(predicted_real, axis=0)
predicted_imag = np.concatenate(predicted_imag, axis=0)

# Combine real and imaginary parts to form complex numbers

predicted_complex = predicted_real + 1j * predicted_imag

# Execution time
# End time
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")



finland_timezone = pytz.timezone('Europe/Helsinki')
current_time =datetime.now(finland_timezone).strftime('%d%m%y')





with open(f'Generated_result/sample135_complex.txt', 'w') as f:
    for i in range(len(predicted_complex)):
        line = []
        for j in range(64):
            real_part = f"{predicted_complex[i, j].real:.4f}"
            imag_part = predicted_complex[i, j].imag
            # If imaginary part is negative, keep the minus sign, else add a plus sign
            imag_str = f"+{imag_part:.4f}" if imag_part >= 0 else f"{imag_part:.4f}"
            line.append(f"{real_part}{imag_str}i")
        f.write("\t".join(line) + "\n")

print("Predicted data saved successfully.")



cuda
Execution time: 0.31540346145629883 seconds
Predicted data saved successfully.
